In [1]:
import numpy as np
from sklearn.cluster import KMeans

In [2]:
embeddings = np.load('embeddings.npy')
print(embeddings.shape)

(202599, 128)


In [3]:
K = 450
clf = KMeans(n_clusters=K, max_iter=350, n_init=5, n_jobs=-1)
clf.fit(embeddings)

/home/anmihajlov/anaconda3/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/home/anmihajlov/anaconda3/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/home/anmihajlov/anaconda3/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/home/anmihajlov/anaconda3/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)
/home/anmihajlov/anaconda3/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=350,
    n_clusters=450, n_init=5, n_jobs=-1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [4]:
with open('data/index/centroids', 'w') as fout:
    for i in range(K):
        cur_centroid = clf.cluster_centers_[i, :]
        print(' '.join(cur_centroid.astype('str')), file=fout)

In [5]:
N, D = embeddings.shape

In [6]:
from collections import defaultdict

def inverse_index(labels):
    result = defaultdict(list)
    for i, label in enumerate(labels):
        result[label].append(i)
        
    return result

In [7]:
centroid2idx = inverse_index(clf.labels_)

with open('data/index/centroid2idx', 'w') as fout:
    for i in range(K):
        print(' '.join(map(str, centroid2idx[i])), file=fout)

In [8]:
import os

filenames = sorted(list(os.listdir(path='data/img_align_celeba/')))

with open('data/index/filenames', 'w') as fout:
    print(' '.join(map(str, filenames)), file=fout)

In [9]:
Q = 512 # descriptor size
random_directions = np.random.randn(D, Q)

with open('data/index/directions', 'w') as fout:
    for i in range(Q):
        print(' '.join(map(str, random_directions[:, i])), file=fout)

In [10]:
descriptors = np.dot(embeddings, random_directions) > 0
print(descriptors.shape)

(202599, 512)


In [11]:
def bool2int(x):
    y = 0
    for i, j in enumerate(x):
        if j:
            y += int(j) << i
            
    return y

def to_binary(values):
    D = len(values)
    result = []
    for i in range(0, D, 64):
        result.append(bool2int(values[i:i + 64]))
    
    return result

In [12]:
with open("data/index/descriptors", 'w') as fout:
    for i in range(N):
        print(' '.join(map(str, to_binary(descriptors[i]))), file=fout)